In [2]:
pip install gym[toy_text]

     ---------------------------------------- 4.8/4.8 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.2
    Uninstalling pygame-2.5.2:
      Successfully uninstalled pygame-2.5.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Toegang geweigerd: 'C:\\Users\\Admin\\anaconda3\\Lib\\site-packages\\~ygame\\base.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



# Loading DiscreteEnv

In [1]:
import gym
from pt1_plot import *

print(gym.__version__)

# We will load a DiscreteEnv and retrieve the probability and reward information
env = gym.make("FrozenLake8x8-v1", desc=None, map_name=None, is_slippery=True)

0.26.2


# Policy computation
env.P[state][action] = [(probability, nextstate, reward, done), ...]

We search for a stationary policy, as evaluation happens on $T=1000$
where it is improbable that the MDP isn't done.

In [2]:
import copy
alpha= 1
states = range(len(env.P)) 

# initializing v,q,policy
v = {state:0 for state in states} 
policy = {state:0 for state in states} # assume that 0 is an action
q = {state:{action:0 for action in range(len(env.P[state])) } for state in states}

#these are for plotting
vv = [] 
pp = []
qq = []

for i in range(300):
    for state in states:
        info = env.P[state][policy[state]]
        # the MC version samples the b, in a simulation we can importance sample b[0]
        # a similar thing can be done with the q function
        v[state] = sum(b[0]*(b[2]+alpha*v[b[1]]) if not(b[3]) else b[0]*b[2] for b in info)

    for state in range(len(env.P)):
        for action in range(len(env.P[state])): 
            val = sum(b[0]*(b[2]+v[b[1]]) for b in env.P[state][action])
            q[state][action]= val

    for state in states:
        ma , mval = 0, q[state][0]
        for action,val in q[state].items():
            if val>mval:
                ma = action 
                mval = val
        policy[state] = ma
    vv.append(v.copy())
    pp.append(policy.copy())
    qq.append(copy.deepcopy(q))

sol = list(pp[-1].items())
print(sol)
print(vv[-1][0])
intvp(vv,pp)

[(0, 1), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 3), (9, 3), (10, 3), (11, 3), (12, 3), (13, 2), (14, 1), (15, 1), (16, 3), (17, 3), (18, 3), (19, 0), (20, 0), (21, 2), (22, 1), (23, 1), (24, 3), (25, 3), (26, 3), (27, 3), (28, 1), (29, 2), (30, 1), (31, 1), (32, 0), (33, 0), (34, 2), (35, 3), (36, 3), (37, 2), (38, 2), (39, 1), (40, 0), (41, 1), (42, 2), (43, 0), (44, 0), (45, 2), (46, 2), (47, 1), (48, 0), (49, 2), (50, 0), (51, 0), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 0), (59, 0), (60, 0), (61, 2), (62, 2), (63, 0)]
0.9999999999995879


interactive(children=(IntSlider(value=0, description='iterations', max=299), Output()), _dom_classes=('widget-…

# Policy evaluation: here's where YOU also code
Insert here your code to evaluate
the total expected rewards over the planning horizon T
if one follows your policy. Do the same for a random policy (i.e. the
sample policy given above). As a sanity check, your policy should get an
expected reward of at least the one obtained by the random policy!

In [3]:
T = 1000  # Given horizon
random_policy = {t: {i: env.action_space.sample()
              for i in range(env.observation_space.n)}
          for t in range(T)}

sol_policy ={t: {i:pp[-1][i] 
              for i in range(env.observation_space.n)}
                for t in range(T)}

def MC_eval(pol,nsim = 10**3):
    # does MC estimation of the expected reward
    tmp = 0
    for _ in range(nsim):
        stat = env.reset()
        total_reward = 0
        state = stat[0]
        for i, t in enumerate(range(T)): #wut i = t ?
            # env.render()
            action = pol[t][state]
            state, reward, done, _, _ = env.step(action)
            total_reward +=reward
            # if the MDP is stuck, we end the simulation here
            if done:
                break
        env.close()
        tmp +=total_reward/nsim
    return tmp

print(f"MC_eval of random policy: {MC_eval(random_policy)}")
print(f"MC_eval of sol policy: {MC_eval(sol_policy)}")

MC_eval of random policy: 0.022000000000000013
MC_eval of sol policy: 1.0000000000000007
